# Naive Bayes para clasificacion de texto

### Modelo multilingues: Se usan datos de todos los idiomas.


Es la tecnica usada para poder calcular 
$$P(texto|subject)$$ 
que sera usada posteriormente con el modelo probabilistico.


Se vectorizan los titulos de las tesis y despues se aplica el clasificador.


In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import pickle
import time

from sklearn.naive_bayes import MultinomialNB

In [2]:
# carpeta donde guardar los modelos
carpeta_modelos = "./Models/Naive_Bayes/"

# carpeta para guardar las predicciones
results_path = "./Predictions/"

# carpeta donde guardar los hyperparametros
hyperparameters_path = "./Models/Hyperparameters/"

# carpeta donde se toman los datos
carpeta_datos = ".\\Data\\Text_classification_data\\"

# nombre archivos de datos
nombre_train = "train_df.csv"
nombre_val = "val_df.csv"

## Cargar los datos

In [3]:
# datos de entrenamiento
df_train = pd.read_csv(carpeta_datos + nombre_train, encoding='utf-8-sig', index_col=0)

# cargar los datos
print(f"Datos de entrenamiento: {df_train.shape[0]}")
df_train.head()

Datos de entrenamiento: 141345


,thesis,subject
287537,Analyzing undergraduate admissions criteria (t...,"91—Game theory, economics, social and behavior..."
123686,Aproximações markovianas e reamostragem para c...,62—Statistics
288977,Imperfect information in spatial elections: An...,"91—Game theory, economics, social and behavior..."
153883,A Refined Gross-Prasad Conjecture for Unitary ...,11—Number theory
297075,Real-time Detection and Suppression of Malicio...,68—Computer science


In [4]:
# tomar los datos de interes, thesis y sibjects
X_train = df_train['thesis']
y_train = df_train['subject']

In [5]:
# cargar los datos val
df_val = pd.read_csv(carpeta_datos + nombre_val, encoding='utf-8-sig', index_col=0)
print(f"Datos de validacion: {df_val.shape[0]}")

# poner en formato correcto
X_val = df_val['thesis']
y_val = df_val['subject']

Datos de validacion: 15705


In [6]:
# combinar el conjunto de entrenamiento y validación
X_combined = np.concatenate((X_train, X_val))
y_combined = np.concatenate((y_train, y_val))

from sklearn.model_selection import PredefinedSplit
# crear un índice para dividir entrenamiento y validación en PredefinedSplit
split_index = [-1] * len(X_train) + [0] * len(X_val)
ps = PredefinedSplit(test_fold = split_index)

### Multinomial Naive Bayes

Primero se vectorizan los datos con CountVectorizer, despues se clasifica con Naive Bayes MultinomialNB

In [7]:
# definir los pasos del pipeline
m_nb_steps = [('vecto', CountVectorizer(stop_words = 'english')),
              ('nb', MultinomialNB())]

# crear el pipeline
m_nb_pipeline = Pipeline(m_nb_steps)

In [8]:
# delimitar los parametros a explorar
parametros_m_nb = {'vecto__ngram_range': [(1, 1), (1, 2)],
                   'vecto__max_features': [None, 30000, 60000],
                   'vecto__min_df': [1, 2, 10],
                   'nb__alpha': [0.001, 0.01, 0.1, 1, 0.05],
                   'nb__fit_prior': [True, False]
                  }

In [9]:
# contar tiempo de ejecucion
start_time = time.time()

# explorar los parametros
grid_search_m_nb = GridSearchCV(m_nb_pipeline,
                                param_grid = parametros_m_nb,
                                scoring = 'f1_macro', verbose=0, 
                                cv=ps,
                                refit = False) # solo entrenar con datos de training

# entrenar
grid_search_m_nb.fit(X_combined, y_combined)

# ver cuanto tardo
end_time = time.time()
elapsed_time = end_time - start_time
print(f"La optimizacion de MultinomialNB duro {round(elapsed_time, 1)} segundos")

La optimizacion de MultinomialNB duro 573.2 segundos


In [10]:
# imprimir los mejores parametros
print('Mejores parametros:', grid_search_m_nb.best_params_)

Mejores parametros: {'nb__alpha': 0.01, 'nb__fit_prior': True, 'vecto__max_features': None, 'vecto__min_df': 1, 'vecto__ngram_range': (1, 2)}


In [11]:
# tomar los resultados de la evaluacion de hyper parametros
results_df = pd.DataFrame(grid_search_m_nb.cv_results_)
# seleccionar solumnas de interes
results_df = results_df[[col for col in results_df.columns if 'param_' in col] + ['mean_test_score']]
# renombrar la columna del puntaje
results_df = results_df.rename(columns={'mean_test_score': 'val_score'})
results_df = results_df.sort_values('val_score', ascending =False)

# guardar
results_df.to_csv(hyperparameters_path + "Naive_Bayes.csv", index=False)

results_df.head()

,param_nb__alpha,param_nb__fit_prior,param_vecto__max_features,param_vecto__min_df,param_vecto__ngram_range,val_score
37,0.01,True,None,1,"(1, 2)",0.455615
55,0.01,False,None,1,"(1, 2)",0.452643
39,0.01,True,None,2,"(1, 2)",0.451998
165,0.05,False,None,2,"(1, 2)",0.448972
1,0.001,True,None,1,"(1, 2)",0.447495


### Entrenar un modelo con los mejores parametros

In [12]:
# tomar los mejores parametros segun el conjunto de validacion
best_params = grid_search_m_nb.best_params_

# crear un nuevo modelo usando estos mejores parámetros
nb_best = m_nb_pipeline.set_params(**best_params)
nb_best

# entrenar el modelo final solo con el conjunto de entrenamiento
nb_best.fit(X_train, y_train)

Pipeline(steps=[('vecto',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('nb', MultinomialNB(alpha=0.01))])

In [13]:
# ver el tamaño del vocabulario, solo por curiosidad
vocab_size = len(nb_best.named_steps['vecto'].vocabulary_)
print(f"Tamaño del vocabulario: {vocab_size}")

Tamaño del vocabulario: 551776


## Ver desempeño

### 1) Desempeño en datos de entrenamiento

In [14]:
# hacer predicciones en los train data con los modelos
y_train_pred_m = nb_best.predict(X_train)

In [15]:
# f1 macro
f1_macro_m = f1_score(y_train, y_train_pred_m, average='macro')
print(f"F1 macro::  {round(f1_macro_m, 5)}")

# f1 weighted
f1_weighted_m = f1_score(y_train, y_train_pred_m, average='weighted')
print(f"F1 weighted:  {round(f1_weighted_m, 5)}")

# f1 micro
f1_micro_m = f1_score(y_train, y_train_pred_m, average='micro')
print(f"F1 micro:  {round(f1_micro_m, 5)}")

F1 macro::  0.96799
F1 weighted:  0.96906
F1 micro:  0.969


### 2) Desempeño en datos de validacion

In [16]:
# hacer predicciones en los test data con los modelos
y_val_pred_m = nb_best.predict(X_val)

In [17]:
# f1 macro
f1_macro_m = f1_score(y_val, y_val_pred_m, average='macro')
print(f"F1 macro::  {round(f1_macro_m, 5)}")

# f1 weighted
f1_weighted_m = f1_score(y_val, y_val_pred_m, average='weighted')
print(f"F1 weighted:  {round(f1_weighted_m, 5)}")

# f1 micro
f1_micro_m = f1_score(y_val, y_val_pred_m, average='micro')
print(f"F1 micro:  {round(f1_micro_m, 5)}")

F1 macro::  0.45562
F1 weighted:  0.64221
F1 micro:  0.64464


### 3) Pruebas manuales

In [18]:
X_ejemplos = np.array(["An Alternative Definition of Stable Models Via Lukasiewicz Logic",
                      "Energy of a graph and Randić index of subgraphs",
                      "Analyzing Mexico City's Airbnb Reviews During COVID-19: A Computational Framework Utilizing BERT Embeddings",
                      "A Probabilistic Model for Node Classification in Directed Graphs"])

# haceer las predicciones
m_ejemplos = nb_best.predict(X_ejemplos)

print("\nPredicciones:")
print(m_ejemplos)


Predicciones:
['03—Mathematical logic and foundations' '05—Combinatorics'
 '62—Statistics' '68—Computer science']


## Guardar modelo

In [19]:
# save MultinomialNB
with open(carpeta_modelos + 'Naive_Bayes.pkl','wb') as f:
    pickle.dump(nb_best,f)

Ejemplo de cargar

In [20]:
# load MultinomialNB
with open(carpeta_modelos + 'Naive_Bayes.pkl','rb') as f:
    nb_best = pickle.load(f)
print(nb_best)

Pipeline(steps=[('vecto',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('nb', MultinomialNB(alpha=0.01))])


## Hacer predicciones para todos los datos

In [21]:
# cargar el df con todas las tesis
df_full = pd.read_csv(carpeta_datos + "full_df.csv", encoding='utf-8-sig', index_col=0, na_values=None, keep_default_na=False)
print(f"Hacer predicciones en {df_full.shape[0]} datos")

# tomar info de interes
X_full = df_full['thesis']

Hacer predicciones en 267774 datos


In [22]:
# hacer predicciones con el mejor modelo
y_full = nb_best.predict(X_full)

# poner como una columna del df
df_full['prediction'] = y_full

# ver
df_full.head()

,thesis,prediction
255897,Spatial Modeling of Activity and User Assistan...,68—Computer science
206917,The Purchasing Power Theories of Trade Unions,"91—Game theory, economics, social and behavior..."
113793,Stroomverdeeling in versterkerbuizen,"78—Optics, electromagnetic theory"
113794,Uitbreiding van electromagnetische golven en e...,"78—Optics, electromagnetic theory"
113795,Spanningsregeling van electrische machines doo...,"78—Optics, electromagnetic theory"


In [23]:
# poner las predicciones en un diccionario
# con formato correcto
predicciones_finales = {v: info_row['prediction']
                       for v, info_row in df_full.iterrows()}

# guardar estas predicciones
with open(results_path + 'Naive_Bayes.pkl', 'wb') as f:
    pickle.dump(predicciones_finales, f)